In [ ]:


import copy
import random

# Constants
WHITE = 'w'
BLACK = 'b'
FILES = 'abcdefghijklmnopqrstuvwxyz'[:8]
RANKS = '12345678'

def coord_from_algebraic(sq: str):
    """e.g. 'e2' -> (row, col) where row 0 is top (rank 8) and row 7 is bottom (rank 1)"""
    if len(sq) != 2:
        return None
    file = sq[0]
    rank = sq[1]
    if file not in FILES[:8] or rank not in RANKS:
        return None
    col = FILES.index(file)
    row = 8 - int(rank)
    return row, col

def algebraic_from_coord(row, col):
    return f"{FILES[col]}{8-row}"

class Piece:
    def __init__(self, kind: str, color: str):
        self.kind = kind.lower()  # 'p','r','n','b','q','k'
        self.color = color

    def __repr__(self):
        return f"{self.color}{self.kind}"

    def symbol(self):
        s = self.kind.upper() if self.color == WHITE else self.kind.lower()
        return s

class Board:
    def __init__(self):
        self.board = [[None]*8 for _ in range(8)]
        self.to_move = WHITE
        self.move_history = []
        self.setup_startpos()

    def setup_startpos(self):
        #pawns
        for c in range(8):
            self.board[6][c] = Piece('p', WHITE)
            self.board[1][c] = Piece('p', BLACK)
        #rooks
        self.board[7][0] = Piece('r', WHITE); self.board[7][7] = Piece('r', WHITE)
        self.board[0][0] = Piece('r', BLACK); self.board[0][7] = Piece('r', BLACK)
        #knights
        self.board[7][1] = Piece('n', WHITE); self.board[7][6] = Piece('n', WHITE)
        self.board[0][1] = Piece('n', BLACK); self.board[0][6] = Piece('n', BLACK)
        #bishops
        self.board[7][2] = Piece('b', WHITE); self.board[7][5] = Piece('b', WHITE)
        self.board[0][2] = Piece('b', BLACK); self.board[0][5] = Piece('b', BLACK)
        #queens
        self.board[7][3] = Piece('q', WHITE)
        self.board[0][3] = Piece('q', BLACK)
        #kings
        self.board[7][4] = Piece('k', WHITE)
        self.board[0][4] = Piece('k', BLACK)

    def in_bounds(self, r, c):
        return 0 <= r < 8 and 0 <= c < 8

    def piece_at(self, r, c):
        if not self.in_bounds(r,c): return None
        return self.board[r][c]

    def all_pieces(self):
        for r in range(8):
            for c in range(8):
                p = self.board[r][c]
                if p:
                    yield (r,c,p)

    def copy(self):
        return copy.deepcopy(self)

    def print_board(self):
        print("  +------------------------+")
        for r in range(8):
            print(8-r, end=" | ")
            for c in range(8):
                p = self.board[r][c]
                if p:
                    print(p.symbol(), end=' ')
                else:
                    print('.', end=' ')
            print("|")
        print("  +------------------------+")
        print("    a b c d e f g h")
        print(f"To move: {'White' if self.to_move == WHITE else 'Black'}")

    def king_position(self, color):
        for r,c,p in self.all_pieces():
            if p.kind == 'k' and p.color == color:
                return r,c
        return None

    def is_square_attacked(self, row, col, by_color):
        """Return True if square (row,col) is attacked by any piece of by_color.
           This function uses piece movement patterns (pseudo-legal attacks)."""
        #Pawns attack differently than they move
        dir = -1 if by_color == WHITE else 1  # white pawns move up (to smaller row index)
        #pawn attacks
        for dc in (-1,1):
            r = row + dir
            c = col + dc
            if self.in_bounds(r,c):
                p = self.piece_at(r,c)
                if p and p.color == by_color and p.kind == 'p':
                    return True
        #knights
        knight_offsets = [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]
        for dr,dc in knight_offsets:
            r,c = row+dr, col+dc
            if self.in_bounds(r,c):
                p = self.piece_at(r,c)
                if p and p.color == by_color and p.kind == 'n':
                    return True
        #sliding pieces (rook/queen) orthogonal
        for dr,dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            r,c = row+dr, col+dc
            while self.in_bounds(r,c):
                p = self.piece_at(r,c)
                if p:
                    if p.color == by_color and (p.kind == 'r' or p.kind == 'q'):
                        return True
                    break
                r += dr; c += dc
        #sliding pieces (bishop/queen) diagonal
        for dr,dc in [(1,1),(1,-1),(-1,1),(-1,-1)]:
            r,c = row+dr, col+dc
            while self.in_bounds(r,c):
                p = self.piece_at(r,c)
                if p:
                    if p.color == by_color and (p.kind == 'b' or p.kind == 'q'):
                        return True
                    break
                r += dr; c += dc
        #king adjacent
        for dr in (-1,0,1):
            for dc in (-1,0,1):
                if dr == 0 and dc == 0: continue
                r,c = row+dr, col+dc
                if self.in_bounds(r,c):
                    p = self.piece_at(r,c)
                    if p and p.color == by_color and p.kind == 'k':
                        return True
        return False

    def generate_pseudo_legal_moves_from(self, row, col):
        """Generate moves for piece at row,col without considering leaving own king in check.
           Returns list of (r_from,c_from,r_to,c_to,promotion_kind_or_None)"""
        p = self.piece_at(row,col)
        if not p: return []
        moves = []
        color = p.color
        if p.kind == 'p':
            direction = -1 if color == WHITE else 1
            start_row = 6 if color == WHITE else 1
            # one forward
            r1, c1 = row + direction, col
            if self.in_bounds(r1,c1) and self.piece_at(r1,c1) is None:
                if r1 == 0 or r1 == 7:
                    moves.append((row,col,r1,c1,'q'))  # default to queen
                else:
                    moves.append((row,col,r1,c1,None))
                #two forward from starting row
                r2 = row + 2*direction
                if row == start_row and self.in_bounds(r2,c1) and self.piece_at(r2,c1) is None:
                    moves.append((row,col,r2,c1,None))
            #captures
            for dc in (-1,1):
                rc, cc = row+direction, col+dc
                if self.in_bounds(rc,cc):
                    target = self.piece_at(rc,cc)
                    if target and target.color != color:
                        if rc == 0 or rc == 7:
                            moves.append((row,col,rc,cc,'q'))
                        else:
                            moves.append((row,col,rc,cc,None))
        elif p.kind == 'n':
            offsets = [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]
            for dr,dc in offsets:
                r,c = row+dr, col+dc
                if not self.in_bounds(r,c): continue
                t = self.piece_at(r,c)
                if t is None or t.color != color:
                    moves.append((row,col,r,c,None))
        elif p.kind in ('b','r','q'):
            directions = []
            if p.kind in ('r','q'):
                directions += [(1,0),(-1,0),(0,1),(0,-1)]
            if p.kind in ('b','q'):
                directions += [(1,1),(1,-1),(-1,1),(-1,-1)]
            for dr,dc in directions:
                r,c = row+dr, col+dc
                while self.in_bounds(r,c):
                    t = self.piece_at(r,c)
                    if t is None:
                        moves.append((row,col,r,c,None))
                    else:
                        if t.color != color:
                            moves.append((row,col,r,c,None))
                        break
                    r += dr; c += dc
        elif p.kind == 'k':
            for dr in (-1,0,1):
                for dc in (-1,0,1):
                    if dr==0 and dc==0: continue
                    r,c = row+dr, col+dc
                    if not self.in_bounds(r,c): continue
                    t = self.piece_at(r,c)
                    if t is None or t.color != color:
                        moves.append((row,col,r,c,None))
        return moves

    def all_legal_moves(self, color):
        """Return list of legal moves for color: (r1,c1,r2,c2,prom)"""
        moves = []
        for r,c,p in self.all_pieces():
            if p.color == color:
                pseudo = self.generate_pseudo_legal_moves_from(r,c)
                for mv in pseudo:
                    # simulate and check king safety
                    board2 = self.copy()
                    board2.make_move(mv, actually_record=False)
                    if not board2.is_in_check(color):
                        moves.append(mv)
        return moves

    def is_in_check(self, color):
        kp = self.king_position(color)
        if kp is None:
            return True  # no king found, treat as in check (shouldn't happen)
        kr,kc = kp
        return self.is_square_attacked(kr,kc, WHITE if color==BLACK else BLACK)

    def make_move(self, move_tuple, actually_record=True):
        """Perform move: (r1,c1,r2,c2,prom) ; does not check legality (caller should)."""
        r1,c1,r2,c2,prom = move_tuple
        piece = self.board[r1][c1]
        captured = self.board[r2][c2]
        self.board[r2][c2] = piece
        self.board[r1][c1] = None
        # handle promotion
        if piece and piece.kind == 'p' and (r2 == 0 or r2 == 7):
            promo_kind = prom if prom else 'q'
            self.board[r2][c2] = Piece(promo_kind, piece.color)
        if actually_record:
            self.move_history.append((move_tuple, captured))
            self.to_move = WHITE if self.to_move == BLACK else BLACK

    def algebraic_to_move(self, s):
        """Parse moves like 'e2e4' or 'e7e8q' -> move tuple or None"""
        s = s.strip().lower()
        if len(s) < 4:
            return None
        from_sq = s[0:2]; to_sq = s[2:4]
        prom = s[4] if len(s) >=5 else None
        from_coord = coord_from_algebraic(from_sq)
        to_coord = coord_from_algebraic(to_sq)
        if from_coord is None or to_coord is None:
            return None
        r1,c1 = from_coord; r2,c2 = to_coord
        p = self.piece_at(r1,c1)
        if p is None or p.color != self.to_move:
            return None
        legal = self.all_legal_moves(self.to_move)
        for mv in legal:
            if mv[0]==r1 and mv[1]==c1 and mv[2]==r2 and mv[3]==c2:
                if mv[4] is not None:
                    if prom in ('q','r','b','n'):
                        return (r1,c1,r2,c2,prom)
                    else:
                        return (r1,c1,r2,c2,mv[4])  # default queen
                return mv
        return None

    def game_status(self):
        """Return ('ongoing'/'checkmate'/'stalemate', winner_or_None)"""
        color = self.to_move
        legal = self.all_legal_moves(color)
        if legal:
            return 'ongoing', None
        if self.is_in_check(color):
            #checkmate
            winner = WHITE if color == BLACK else BLACK
            return 'checkmate', winner
        else:
            return 'stalemate', None

#Simple random AI
def random_ai_move(board: Board):
    legal = board.all_legal_moves(board.to_move)
    if not legal: return None
    return random.choice(legal)

def pretty_move_str(mv):
    r1,c1,r2,c2,prom = mv
    s = f"{algebraic_from_coord(r1,c1)}{algebraic_from_coord(r2,c2)}"
    if prom:
        s += prom
    return s

def main():
    print("Simple CLI Chess — learning project")
    b = Board()
    mode = ''
    while mode not in ('1','2'):
        mode = input("Choose mode: 1) 2-player local  2) Play vs random-AI : ").strip()
    vs_ai = (mode == '2')
    human_color = WHITE
    if vs_ai:
        who = ''
        while who not in ('w','b'):
            who = input("Play as (w)hite or (b)lack? [w]: ").strip().lower() or 'w'
        human_color = WHITE if who == 'w' else BLACK

    while True:
        b.print_board()
        status, winner = b.game_status()
        if status == 'checkmate':
            print("Checkmate! Winner:", "White" if winner==WHITE else "Black")
            break
        elif status == 'stalemate':
            print("Stalemate! It's a draw.")
            break
        else:
            if b.is_in_check(b.to_move):
                print("CHECK!")

        if vs_ai and b.to_move != human_color:
            print("AI thinking...")
            mv = random_ai_move(b)
            if mv is None:
                print("No legal moves for AI.")
                continue
            b.make_move(mv)
            print("AI plays:", pretty_move_str(mv))
            continue

        #human turn
        prompt = f"{'White' if b.to_move==WHITE else 'Black'} move (e.g. e2e4), or 'resign'/'undo': "
        s = input(prompt).strip()
        if s.lower() in ('resign','quit','exit'):
            print(f"{'White' if b.to_move==WHITE else 'Black'} resigned. Winner:", "Black" if b.to_move==WHITE else "White")
            break
        if s.lower() == 'undo':
            if not b.move_history:
                print("Nothing to undo.")
                continue
            #undo last move
            last_move, captured = b.move_history.pop()
            r1,c1,r2,c2,prom = last_move
            moved_piece = b.board[r2][c2]
            if prom is not None and moved_piece and moved_piece.kind != 'p':
                moved_piece = Piece('p', moved_piece.color)
            b.board[r1][c1] = moved_piece
            b.board[r2][c2] = captured
            b.to_move = WHITE if b.to_move == BLACK else BLACK
            continue

        mv = b.algebraic_to_move(s)
        if mv is None:
            print("Invalid move or illegal. Use format 'e2e4'.")
            continue
        b.make_move(mv)
    print("Game over. Thanks for playing!")

if __name__ == '__main__':
    main()
